In [1]:
import SoapySDR
from SoapySDR import *
import numpy as np
import signal


In [2]:
args = dict(driver="hackrf")
sdr = SoapySDR.Device(args)

print("Antennas: ", sdr.listAntennas(SOAPY_SDR_RX, 0))
print("Gains: ", sdr.listGains(SOAPY_SDR_RX, 0))
print("Min/Max Freqs: ")
freqs = sdr.getFrequencyRange(SOAPY_SDR_RX, 0)
for freq in freqs:
    print(freq)


Antennas:  ('TX/RX',)
Gains:  ('LNA', 'AMP', 'VGA')
Min/Max Freqs: 
0, 7.25e+09


In [3]:
# receive example
rxStream = sdr.setupStream(SOAPY_SDR_RX, SOAPY_SDR_CF32)
sdr.activateStream(rxStream)
buff = np.array([0]*1024, np.complex64)
sr = sdr.readStream(rxStream, [buff], len(buff))
print("IQ data (first 2 samples): ", buff[0:2])
print("num samples or error code: ", sr.ret)
print("flags set by receive operation: ", sr.flags)
print("timestamp for receive buffer: ", sr.timeNs)
sdr.deactivateStream(rxStream)
sdr.closeStream(rxStream)

IQ data (first 2 samples):  [0.01574803+0.00787402j 0.00787402+0.00787402j]
num samples or error code:  1024
flags set by receive operation:  0
timestamp for receive buffer:  0


In [4]:
state = dict(running=True)
def signal_handler(signum, _):
        print('Signal handler called with signal {}'.format(signum))
        state['running'] = False

signal.signal(signal.SIGINT, signal_handler)

<function _signal.default_int_handler>

In [5]:
tx_chan = 0
tx_sps = 10e6       # 10MHz  sampling rate
freq = 900e6   # 900Mhz center frequency
tx_gain = 0         # 0db


sdr.setSampleRate(SOAPY_SDR_TX, tx_chan, tx_sps)
sdr.setFrequency(SOAPY_SDR_TX, tx_chan, freq)
sdr.setGain(SoapySDR.SOAPY_SDR_TX, tx_chan, tx_gain)

tx_stream = sdr.setupStream(SOAPY_SDR_TX, SOAPY_SDR_CF32, [tx_chan])
sdr.activateStream(tx_stream)

tx_data = [1 + 0j] * 1024

while state["running"]:
    sdr.writeStream(SoapySDR.SOAPY_SDR_TX, [tx_chan], [tx_data], len(tx_data))

sdr.deactivateStream(tx_stream)
sdr.closeStream(tx_stream)

TypeError: in method 'Device_writeStream__', argument 2 of type 'SoapySDR::Stream *'

In [ ]:
# import SoapySDR

# # Open the SDR device
# sdr = SoapySDR.Device(dict(driver="your_device_driver"))

# # Set the transmit sample rate
# sdr.setSampleRate(SoapySDR.SOAPY_SDR_TX, 10e6)

# # Set the transmit center frequency
# freq = 900e6
# sdr.setFrequency(SoapySDR.SOAPY_SDR_TX, 0, freq)

# # Set the transmit gain
# gain = 30
# sdr.setGain(SoapySDR.SOAPY_SDR_TX, 0, gain)

# # Create a buffer of complex samples to transmit
# num_samples = 1000
# samples = [complex(1.0, 0.0)] * num_samples

# # Transmit the samples
# sdr.writeStream(
#     SoapySDR.SOAPY_SDR_TX, # direction: TX
#     0,                     # channel: 0
#     samples,               # buffer of samples to transmit
#     len(samples),          # number of samples to transmit
#     timeoutUs=int(1e6),    # timeout in microseconds (optional)
# )

# # Close the SDR device
# sdr.close()